In [1]:
import os
import json
import shutil
from pathlib import Path
import unicodedata
import re

In [8]:
PARENT_DIR = '/home/jared/sc2reader/data/replays'
ARCHIVE_PATH = 'C:/Users/jared/Desktop/Archive'
REPLAY_EXT = '.SC2Replay'

In [6]:
#https://stackoverflow.com/questions/9618862/how-to-parse-a-directory-structure-into-dictionary

def set_leaf(tree, branches, leaf):
    """ Set a terminal element to *leaf* within nested dictionaries.              
    *branches* defines the path through dictionaries.                            

    Example:                                                                      
    >>> t = {}                                                                    
    >>> set_leaf(t, ['b1','b2','b3'], 'new_leaf')                                 
    >>> print t                                                                   
    {'b1': {'b2': {'b3': 'new_leaf'}}}                                             
    """
    if len(branches) == 1:
        tree[branches[0]] = leaf
        return

    if not branches[0] in tree:
        tree[branches[0]] = {}
    
    set_leaf(tree[branches[0]], branches[1:], leaf)



In [4]:
def make_directory_map(startpath):

    tree = {}
    for root, dirs, files in os.walk(startpath):
        branches = [startpath]
        if root != startpath:
            branches.extend(os.path.relpath(root, startpath).split('/'))

        set_leaf(tree, branches, dict([(d,{}) for d in dirs]+ \
                                    [(f,None) for f in files]))

    return tree

In [12]:
def move_zip_files(startpath):
    for root, dirs, files in os.walk(startpath):
        for name in files:
            if '.zip' in name:
                archive_folder = os.path.split(root)[1] # get bottom folder name
                new_path = os.path.join(ARCHIVE_PATH, archive_folder, name)
                new_folder = Path(os.path.split(new_path)[0])
                # check if new_path exists, create if it doesn't
                if not os.path.isdir(new_folder):
                    new_folder.mkdir(parents=True)
                current_path = os.path.join(root, name)
                shutil.move(current_path, new_path)

In [2]:
# Loop through dict and extract all zipped files to folders of the same name
def file_unzipper(map_dict, root):
    for key, value in map_dict.items():
        if isinstance(value, dict):
            # recurse into unzipper, keep track of path
            try:
                file_unzipper(value, os.path.join(root, key))
            except:
                print(root, key)
        else:
            if '.zip' in key:
                path = os.path.join(root, key)
                zipless_name = os.path.splitext(key)[0]
                new_folder = os.path.join(root, zipless_name)
                try:
                    shutil.unpack_archive(path, new_folder)
                except:
                    print(path)

In [10]:
def slugify(value, allow_unicode=False):
    """
    Taken from https://github.com/django/django/blob/master/django/utils/text.py
    Convert to ASCII if 'allow_unicode' is False. Convert spaces or repeated
    dashes to single dashes. Remove characters that aren't alphanumerics,
    underscores, or hyphens. Convert to lowercase. Also strip leading and
    trailing whitespace, dashes, and underscores.
    """
    value = str(value)
    if allow_unicode:
        value = unicodedata.normalize('NFKC', value)
    else:
        value = unicodedata.normalize('NFKD', value).encode('ascii', 'ignore').decode('ascii')
    value = re.sub(r'[^\w\s-]', '', value.lower())
    return re.sub(r'[-\s]+', '-', value).strip('-_')

In [9]:
def despace_names(startpath):
    for root, dirs, files in os.walk(startpath):
        for name in files:
            # loop through each file name and then replace space with _
            old_name = os.path.join(root, name)
            reg_split = re.compile('{}'.format(REPLAY_EXT), flags=re.I)
            name = reg_split.split(name)
            if len(name) == 0: # file ext was not found
                continue
            
            new_name = slugify(name[0])+REPLAY_EXT
            new_name = os.path.join(root, new_name)
            os.rename(old_name, new_name)

In [15]:
# unzip all files
file_unzipper(make_directory_map(PARENT_DIR), '')

# move all zip files into archive folder
move_zip_files(PARENT_DIR)

# ensure correct naming of files
despace_names(PARENT_DIR)

#re-make directory map after unpacking and moving
tree = make_directory_map(PARENT_DIR)
# write final directory map to json file
with open('../info/file_map.json', 'w', encoding="utf-8") as file:
        json.dump(tree, file, indent=4, separators=(',', ': '))

/home/jared/sc2reader/data/replays/SpawningTool/Pro/page960.zip
/home/jared/sc2reader/data/replays/SpawningTool/Pro/page600.zip
/home/jared/sc2reader/data/replays/SpawningTool/Pro/page550.zip


FileNotFoundError: [Errno 2] No such file or directory: '/home/jared/archive/Other/page549.zip'

In [36]:
#re-make directory map after unpacking and moving
tree = make_directory_map(PARENT_DIR)
# write final directory map to json file
with open('../info/file_map.json', 'w', encoding="utf-8") as file:
        json.dump(tree, file, indent=4, separators=(',', ': '))

In [7]:
file_unzipper(make_directory_map('D:/Apps/Sc2Replays/SpawningTool'), '')

D:/Apps/Sc2Replays/SpawningTool\Other\page1.zip
D:/Apps/Sc2Replays/SpawningTool\Other\page1000.zip
D:/Apps/Sc2Replays/SpawningTool\Other\page1005.zip
D:/Apps/Sc2Replays/SpawningTool\Other\page242.zip
D:/Apps/Sc2Replays/SpawningTool\Other\page345.zip
D:/Apps/Sc2Replays/SpawningTool\Other\page423.zip
D:/Apps/Sc2Replays/SpawningTool\Other\page469.zip
D:/Apps/Sc2Replays/SpawningTool\Other\page499.zip
D:/Apps/Sc2Replays/SpawningTool\Other\page505.zip
D:/Apps/Sc2Replays/SpawningTool\Other\page518.zip
D:/Apps/Sc2Replays/SpawningTool\Other\page538.zip
D:/Apps/Sc2Replays/SpawningTool\Other\page539.zip
D:/Apps/Sc2Replays/SpawningTool\Other\page541.zip
D:/Apps/Sc2Replays/SpawningTool\Other\page617.zip
D:/Apps/Sc2Replays/SpawningTool\Other\page632.zip
D:/Apps/Sc2Replays/SpawningTool\Other\page649.zip
D:/Apps/Sc2Replays/SpawningTool\Other\page75.zip
D:/Apps/Sc2Replays/SpawningTool\Other\page821.zip
D:/Apps/Sc2Replays/SpawningTool\Other\page822.zip
D:/Apps/Sc2Replays/SpawningTool\Other\page823.zip
D

In [13]:
move_zip_files('D:/Apps/Sc2Replays/SpawningTool')

despace_names('D:/Apps/Sc2Replays/SpawningTool')

FileExistsError: [WinError 183] Cannot create a file when that file already exists: 'D:/Apps/Sc2Replays/SpawningTool\\Other\\page101\\iGXY v 小坚强' -> 'D:/Apps/Sc2Replays/SpawningTool\\Other\\page101\\igxy-v.SC2Replay'